In [ ]:
!pip install "deeplake<4"
!pip install torch torchvision deeplake onnx onnxruntime

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import deeplake
import numpy as np
from torch.utils.data import Dataset, DataLoader
print("✅ Dependencies Imported Successfully")

✅ Dependencies Imported Successfully


In [ ]:
# Load dataset
print("Loading MURA dataset...")
train_ds = deeplake.load("hub://activeloop/mura-train")
val_ds = deeplake.load("hub://activeloop/mura-val")
print("✅ MURA Dataset Loaded Successfully")

Loading MURA dataset...


Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/mura-train



|

...

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/mura-val



|

hub://activeloop/mura-val loaded successfully.

✅ MURA Dataset Loaded Successfully


In [ ]:
from PIL import Image

import torchvision.transforms as transforms

# ✅ Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert 1-channel grayscale to 3-channel RGB
    transforms.ToTensor(),
])

class MURADataset(Dataset):
    def __init__(self, ds, transform=None):
        self.ds = ds
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        img = np.array(self.ds[idx]["images"].numpy())  # Convert image tensor to NumPy

        # Convert NumPy array to PIL Image
        img = Image.fromarray(img.astype('uint8'))

        # `study_type` as the label
        label = self.ds[idx]["study_type"].numpy()[0]

        if self.transform:
            img = self.transform(img)  # ✅ Apply transformations

        # Debugging: Print shape of transformed image
        if idx < 5:
            print(f"✅ Sample {idx} - Label: {label}, Image Shape: {img.shape}")

        return img, torch.tensor(label, dtype=torch.long)

subset_size = 500
train_dataset = torch.utils.data.Subset(train_dataset, range(subset_size))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = MURADataset(val_ds, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("✅ Dataset successfully loaded with `study_type` labels!")

✅ Dataset successfully loaded with `study_type` labels!


In [ ]:
print("Loading Pretrained DenseNet-169...")
model = models.densenet169(pretrained=True)

# Modifies classifier for binary classification
model.classifier = nn.Linear(1664, 2)  # Output = 2 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("✅ DenseNet-169 Loaded and Modified for Binary Classification")
print("Dataset structure:")
print(train_ds.tensors)  # This prints available tensors in the dataset

for i in range(5):
    print(f"Sample {i}:")
    print("Study:", train_ds[i]["study"].numpy())
    print("Study Type:", train_ds[i]["study_type"].numpy())  # Potential fracture label?
    print("Region:", train_ds[i]["region"].numpy())  # Body region
    print("Patient ID:", train_ds[i]["patient_id"].numpy())  # Patient ID
    print("-------------------------")



Loading Pretrained DenseNet-169...
✅ DenseNet-169 Loaded and Modified for Binary Classification
Dataset structure:
{'images': Tensor(key='images'), 'study': Tensor(key='study'), 'study_type': Tensor(key='study_type'), 'patient_id': Tensor(key='patient_id'), 'region': Tensor(key='region')}
Sample 0:
Study: [0]
Study Type: [0]
Region: [0]
Patient ID: ['patient09803']
-------------------------
Sample 1:
Study: [0]
Study Type: [0]
Region: [0]
Patient ID: ['patient09803']
-------------------------
Sample 2:
Study: [0]
Study Type: [1]
Region: [0]
Patient ID: ['patient07455']
-------------------------
Sample 3:
Study: [0]
Study Type: [1]
Region: [0]
Patient ID: ['patient07455']
-------------------------
Sample 4:
Study: [0]
Study Type: [1]
Region: [0]
Patient ID: ['patient07107']
-------------------------


In [ ]:
# Loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

EPOCHS = 3  # Reduced to 3 for debugging

print("✅ Training Setup Complete, Starting Training...")

scaler = torch.cuda.amp.GradScaler()  # Enable mixed precision

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():  # Apply mixed precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{EPOCHS}, Batch {i}, Loss: {loss.item()}")

    print(f"✅ Epoch {epoch+1} Completed, Avg Loss: {running_loss/len(train_loader)}")

print("✅ Training Complete!")


✅ Training Setup Complete, Starting Training...


<ipython-input-12-7764bbbdfe1a>:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Enable mixed precision
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


✅ Sample 2 - Label: 1, Image Shape: torch.Size([3, 224, 224])


<ipython-input-12-7764bbbdfe1a>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Apply mixed precision
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 1/3, Batch 0, Loss: 0.6546863913536072
✅ Sample 0 - Label: 0, Image Shape: torch.Size([3, 224, 224])
Epoch 1/3, Batch 10, Loss: 0.603295087814331
Epoch 1/3, Batch 20, Loss: 0.6069369912147522
✅ Sample 1 - Label: 0, Image Shape: torch.Size([3, 224, 224])
✅ Sample 3 - Label: 1, Image Shape: torch.Size([3, 224, 224])
✅ Sample 4 - Label: 1, Image Shape: torch.Size([3, 224, 224])
Epoch 1/3, Batch 30, Loss: 0.6195724606513977
✅ Epoch 1 Completed, Avg Loss: 0.5830188756808639
Epoch 2/3, Batch 0, Loss: 0.3689805567264557
✅ Sample 1 - Label: 0, Image Shape: torch.Size([3, 224, 224])
Epoch 2/3, Batch 10, Loss: 0.2651657164096832
✅ Sample 2 - Label: 1, Image Shape: torch.Size([3, 224, 224])
✅ Sample 3 - Label: 1, Image Shape: torch.Size([3, 224, 224])
Epoch 2/3, Batch 20, Loss: 0.2175253927707672
✅ Sample 0 - Label: 0, Image Shape: torch.Size([3, 224, 224])
✅ Sample 4 - Label: 1, Image Shape: torch.Size([3, 224, 224])
Epoch 2/3, Batch 30, Loss: 0.4182642698287964
✅ Epoch 2 Completed, Avg Lo